# 9.4: Inspecting the `decode_test_dir` and `experiment` directories

`run_test.sh` will generate a new directory called `decode_test_dir` inside the appropriate `exp` directory.

In [ ]:
ls exp/monophones/decode_test_dir

In [ ]:
ls exp/triphones/decode_test_dir

In [ ]:
ls exp/triphones_lda/decode_test_dir

We also saved some "extra" things in a directory called `default_experiment_*` in `exp/experiments` along with an entire copy of the `decode_test_dir`.  

In [ ]:
ls exp/experiments/default_experiment_monophones

In [ ]:
ls exp/experiments/default_experiment_triphones

In [ ]:
ls exp/experiments/default_experiment_triphones_lda

So we will look at the contents of *one* `experiment` dir: `default_experiment_monophones`.

## `results.txt`

This is a simple `.txt` file containing the `word error rate` (`WER`) and `sentence error rate` (`SER`) from our experiment.

`WER` = the percentage of words that were transcribed **incorrectly**.
`SER` = the percentage of audio samples with **at least one** error in them.

In [ ]:
cat exp/experiments/default_experiment_monophones/results.txt

The scoring of `WER` is an application of the `minimum edit distance` algorithm.  You can see some slides on this algorithm [here](https://web.stanford.edu/class/cs124/lec/med.pdf).

### `minimum edit distance`

In this algorithm, there are **three** kinds of errors (for the sake of `kaldi`'s scoring, they are all weighted **equally**, though this can be modified):

 - substitution: replacing the correct word with a new word
     - gold: `The cat is sleeping.`
     - predicted: `The cat is yawning.`
 - insertion: inserting an **extra** word
     - gold: `The cat is sleeping.`
     - predicted: `The big cat is sleeping.`
 - deletion: removing a word
     - gold: `The cat is sleeping.`
     - predicted: `The cat sleeping.`
     
To the right of the `WER`, you'll see some numbers in brackets identifying the number of each type of error made.

```
[total_num_errors / total words, num_insertions, num_deletions, num_substitutions]
```

**Note**: You see `[PARTIAL]` because we only submitted a **portion** of all of the test audio samples for scoring (remember that we did this in `9.0` to reduce the time of decoding).

You'll also notice **5** different scores, each with a different `int` in `wer_*`.  This `int` represents the weight given to the language model in the `HCLG.fst`.  

### `language model weight` (`lmwt`) v. `acoustic model weight` (`acwt`)

It's not the most intuitive way of presenting it, but `kaldi` has the `lmwt` set as a `int`.  If `lmwt=10`, that means you want to weight the language model `10x` more than the acoustic model.  If `lmwt=5`, then you want to weight the language model `5x` more than the `acoustic model`.  Interestingly enough that means that the *lowest* you can weight the language model is `lmwt=1`, where the language model and acoustic model are weighed evenly.

In all of the default scripts in `kaldi`, `lmwt` is set to `10`, implying a *very* heavy weighting of the language model.  

One of the `hyperparameters` you can set in `kaldi_config.json` is `lmwt`.  Our script then takes `+/- 2` more weights and scores all *five* so you can see how much of an impact the weighting has on results.

**Note**: Next week, you will have the opportunity to experiment with different weights.

## `predicted_transcripts_10.txt`

This file is a printout of all of the predicted transcripts, but **only** for the scenario where `lmwt=10`.  

In [ ]:
head exp/experiments/default_experiment_monophones/predicted_transcripts_10.txt

Remember that the **gold** transcripts are located at `raw_data/librispeech-transcripts.txt`.

In [ ]:
head raw_data/librispeech-transcripts.txt

If you wanted to compare a **predicted** transcript for a particular audio sample with its **gold** transcript, you can use the following commands.

In [ ]:
id=1089-134686-0000    # choose the audio sample's utterance id
predicted=exp/experiments/default_experiment_monophones/predicted_transcripts_10.txt  # choose which predicted file

gold=raw_data/librispeech-transcripts.txt
diff <(cat ${gold} | grep ${id}) <(cat ${predicted} | grep ${id})

The **top** output is the **gold** and the **bottom** output is the **predicted**.

In the example above, you can see that the errors made were understandable:  `"stupor"` sounds a lot like `"stew for"` and `"fat mutton pieces"` isn't too far off from `"that month ceases"`.

## `decode_test_dir`

The `decode_test_dir` is a complete copy of the directory generated by `run_test.sh`.  It will contain the same types of files regardless of which acoustic model we used, so again we'll just look at the `monophones` directory for simplicity. 

In [ ]:
ls exp/experiments/default_experiment_monophones/decode_test_dir

### `num_jobs`

There will often be a `num_jobs` file in `kaldi` directories.  This is simply one `integer`, the number of threads used if parallelization was used.

In [ ]:
cat exp/experiments/default_experiment_monophones/decode_test_dir/num_jobs

### `runtime`

This file captured the **time** it took to decode.  It is in the format of `MM:SS`, and we will use this more next week as we try to optimize for **both** performance and speed.

In [ ]:
cat exp/experiments/default_experiment_monophones/decode_test_dir/runtime

### `wer_*`

These files are simply the output of the scoring script, whose results we have accumulated in `results.txt`.

In [ ]:
cat exp/experiments/default_experiment_monophones/decode_test_dir/wer_10

### `lat.*.gz`

You will see a number of `lat.*.gz` files, equal to `num_jobs`.  These are the actual lattices generated during prediction time.

#### lattices

A `lattice` can just be seen as a `graph`.  

In this case, it is a **subset** of the `HCLG.fst` collapsed to the word level.  It shows a number of possible paths that can be used to decode an audio sample, and its size depends on two hyperparameters in `kaldi_config.json`: `max_active` and `beam`, both of which we'll look at in more detail next week.  The **weights** on these arcs is set by the `lmwt`.

Once the `lattice` is generated, the algorithm will simply look for the "shortest path" and return that as the prediction.

There are a few visualizations of the `lattice` in `resource_files/lattices`.

In [ ]:
ls resource_files/lattices
ls resource_files/lattices/mono
ls resource_files/lattices/tri_lda

Below is the `lattice` generated for **one** audio sample: `1089-134686-0000`.

![](resource_files/lattices/mono/mono.svg "mono lattice")

And as you can see, it's huuuuuuge.  Below is zoomed in portion of the `lattice above`.

![mono_lattice_zoomed](resource_files/lattices/mono/mono_zoomed.png)

It's still hard to make out the `arc`s because of the image quality, but each `arc` contains a word and a weight.

Part of the reason we have such a large `lattice` is based on the `hyperparameter`s that we used to build the `lattice`, but it's also a sign of the uncertainty in the model.  In other words, the less certain the model is about what it heard, the more paths it has to include in the `lattice`.

Compare the `lattice`s above (which are from the `monophone` model that had a `WER` of about `55%`) with a `lattice` for the same except audio sample, but generated from the `triphone_lda` model (which had a `WER` of about `30%`).

![](resource_files/lattices/tri_lda/tri_lda.svg "tri_lda lattice")

![tri_lda_lattice_zoomed](resource_files/lattices/tri_lda/tri_lda_zoom.png)

### `decode_test_dir/scoring`

This directory contains some intermediate items needed for scoring.

In [ ]:
ls exp/experiments/default_experiment_monophones/decode_test_dir/scoring

#### `test_filt.txt`

This file contains the **gold** transcripts for all audio samples in the test directory, with some things removed.  In general `kaldi` chooses not to score predictions on `[NOISE]` or `[LAUGHTER]`, etc, and so this file has removed them from the original **gold** transcript file.  In our case, this is irrelevant as we don't have any examples of those "words" in our dataset.

#### `*.tra`

These files contain the prediction for each audio sample.  They are the result of finding the shortest path through the `lattice`s explained above.

In [ ]:
head exp/experiments/default_experiment_monophones/decode_test_dir/scoring/10.tra

You should recognize that the first item on each line is the `utterance_id` of the audio sample, but instead of words, we see numbers.  These numbers correspond to the indices found in `graph/words.txt`.  We can use `utils/int2sym.pl` to convert those indices to the words.

In [ ]:
head exp/experiments/default_experiment_monophones/decode_test_dir/scoring/10.tra | \
    utils/int2sym.pl -f 2- \
    exp/monophones/graph/words.txt

Having converted the numbers to words, `10.tra` would end up as **exactly the same** file as `predicted_transcripts_10.txt` that we saw above.

#### `decode_test_dir/scoring/log`

The `scoring` directory contains its own `log`s.

In [ ]:
ls exp/experiments/default_experiment_monophones/decode_test_dir/scoring/log

The `best_path.*.log`s contain the output of the `C++` commands responsible for finding the best path through the `lattice`.

In [ ]:
head exp/experiments/default_experiment_monophones/decode_test_dir/scoring/log/best_path.10.log

In this case, you can see **three** commands were used:

 - `lattice-scale` which sets the weights on the `lattice` (`inv-acoustic-scale` == `lmwt`)
 - `lattice-best-path` which finds the best path
 - `lattice-add-penalty` which can add a penalty for word-insertions.  Note that in this case the `word-ins-penalty` is set to `0.0`, so no adjustment is made.

The `score.*.log`s simple show the command sent to `compute-wer` to determine the `WER`.

In [ ]:
cat exp/experiments/default_experiment_monophones/decode_test_dir/scoring/log/score.10.log

### `decode_test_dir/log`

The `decode_test_dir` also has its own `log`s.



In [ ]:
ls exp/experiments/default_experiment_monophones/decode_test_dir/log

The only ones that are of particular interest to us are `analyze_alignments.log` and `analyze_lattice_depth_stats.log`.  These two `log`s are the output of a particular analysis script that we called during `run_test.sh`.  

#### `analyze_alignments.log`

This `log` identifies a few statistics for **each** phone:

 - the percent of time this phone appears in the `lattice` predictions
    - `"Overall, * accounts for *% of phone occurences..."`
 - the median, mean, and 95th percentile for the number of frames this phone "required"
    - `"with duration (median, mean, 95-percentile) is (*, *, *) frames"`

In [ ]:
head exp/experiments/default_experiment_monophones/decode_test_dir/log/analyze_alignments.log

You'll also notice that it does the breakdown for each phone in `BIES` notation as well.

In [ ]:
tail exp/experiments/default_experiment_monophones/decode_test_dir/log/analyze_alignments.log

There are also some particular statistics about `SIL`.  

In [ ]:
cat exp/experiments/default_experiment_monophones/decode_test_dir/log/analyze_alignments.log | grep SIL

The assumption we make is that each audio sample (`utterance`) begins and ends with silence and so we'd expect `SIL` to be the first and last phone predicted in each one.  This `log` identifies how often that is the case:

`"At utterance {begin, end}..."`

#### `analyze_lattice_depth_stats.log`

This `log` is similar to the one above, but this one focuses on the "depth" of the `lattice`.

In [ ]:
head exp/experiments/default_experiment_monophones/decode_test_dir/log/analyze_lattice_depth_stats.log

It's unclear what this means exactly, but the `kaldi` documentation defines `lattice depth` as "the average number of arcs that cross a frame".  However, one thing that is clear is that the `lattice`s generated by the `monophone` model are **much larger** than those generated by the `triphone_lda` model (as we saw above).  And the same can be seen here if we compare the `lattice depth`s in the `monophone` model to those in the `triphone_lda` model (below).

In [ ]:
head exp/experiments/default_experiment_triphones_lda/decode_test_dir/log/analyze_lattice_depth_stats.log